# Notes
- this adds a preprocessed (snapped) Beehive bikeways network to the existing and planned networks

In [35]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.pivot_table(df, values='a', index='b', columns='c', aggfunc='sum', fill_value=0)
# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [36]:
# # fill NA values in Spatially enabled dataframes (ignores SHAPE column)
# def fill_na_sedf(df_with_shape_column, fill_value=0):
#     if 'SHAPE' in list(df_with_shape_column.columns):
#         df = df_with_shape_column.copy()
#         shape_column = df['SHAPE'].copy()
#         del df['SHAPE']
#         return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
#     else:
#         raise Exception("Dataframe does not include 'SHAPE' column")

# Setup Outputs directories

In [37]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "bb_modeling_20240911.gdb"]
gdb = os.path.join(outputs[0], outputs[1])



if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])


In [38]:
planned_bb = pd.DataFrame.spatial.from_featureclass('E:\Tasks\Bike_Modeling_Data_Update\Processing\Outputs\BPA_PLANNED_BB.shp')
base_bb = pd.DataFrame.spatial.from_featureclass('E:\Tasks\Bike_Modeling_Data_Update\Processing\Outputs\BPA_BB.shp')
template = pd.DataFrame.spatial.from_featureclass('E:\Tasks\Bike_Modeling_Data_Update\Processing\Outputs\mmn_beehive_modeling.gdb\mmn_with_bb_and_planned')
bb_new_routes = pd.DataFrame.spatial.from_featureclass('E:\Tasks\Bike_Modeling_Data_Update\Processing\Inputs\Beehive_Bikeways_New_Routes_Spplit.shp')

c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [39]:
bb_new_routes['BIKE_L'] = bb_new_routes['TypeCode']
bb_new_routes['BIKE_R'] = bb_new_routes['TypeCode']
bb_new_routes['BikeNetwork'] = 'Y'
bb_new_routes['BB'] = 1
bb_new_routes['CartoCode'] = '3 - Paved Shared Use'
bb_new_routes = bb_new_routes[['BIKE_L', 'BIKE_R', 'BikeNetwork', 'BB', 'CartoCode', 'SHAPE']]
bb_new_routes.head()

,BIKE_L,BIKE_R,BikeNetwork,BB,CartoCode,SHAPE
0,PP,PP,Y,1,3 - Paved Shared Use,"{""paths"": [[[415366, 4595921.699999999], [4153..."
1,PP,PP,Y,1,3 - Paved Shared Use,"{""paths"": [[[413009.0999999996, 4583359.5], [4..."
2,PP,PP,Y,1,3 - Paved Shared Use,"{""paths"": [[[413016.53029999975, 4583031.12859..."
3,PP,PP,Y,1,3 - Paved Shared Use,"{""paths"": [[[412984.5220999997, 4582361.145500..."
4,PP,PP,Y,1,3 - Paved Shared Use,"{""paths"": [[[412976.2999999998, 4582116.5], [4..."


In [40]:
base_bb.loc[base_bb['TypeCode'].isin(['1','3C', 'PP']), 'BIKE_R'] = base_bb['TypeCode']
base_bb.loc[base_bb['TypeCode'].isin(['1','3C', 'PP']), 'BIKE_L'] = base_bb['TypeCode']
planned_bb.loc[base_bb['TypeCode'].isin(['1','3C', 'PP']), 'BIKE_R'] = base_bb['TypeCode']
planned_bb.loc[base_bb['TypeCode'].isin(['1','3C', 'PP']), 'BIKE_L'] = base_bb['TypeCode']

In [41]:
base_bb = base_bb[['Name', 'Oneway', 'Speed', 'AutoNetwor', 'BikeNetwor',
       'PedNetwork', 'SourceData', 'DriveTime', 'BikeTime', 'Pedestrian',
       'Length_Mil', 'ConnectorN', 'CartoCode', 'AADT', 'AADT_YR',
       'BIKE_L', 'BIKE_R', 'VERT_LEVEL', 'BB', 'SHAPE']].copy()

base_bb.columns = ['Name', 'Oneway', 'Speed', 'AutoNetwork', 'BikeNetwork',
       'PedNetwork', 'SourceData', 'DriveTime', 'BikeTime', 'PedestrianTime',
       'Length_Miles', 'ConnectorNetwork', 'CartoCode', 'AADT', 'AADT_YR',
       'BIKE_L', 'BIKE_R', 'VERT_LEVEL', 'BB', 'SHAPE']

In [42]:
planned_bb = planned_bb[['Name', 'Oneway', 'Speed', 'AutoNetwor', 'BikeNetwor',
       'PedNetwork', 'SourceData', 'DriveTime', 'BikeTime', 'Pedestrian',
       'Length_Mil', 'ConnectorN', 'CartoCode', 'AADT', 'AADT_YR',
       'BIKE_L', 'BIKE_R', 'VERT_LEVEL', 'PLANNED', 'SHAPE']].copy()

planned_bb.columns = ['Name', 'Oneway', 'Speed', 'AutoNetwork', 'BikeNetwork',
       'PedNetwork', 'SourceData', 'DriveTime', 'BikeTime', 'PedestrianTime',
       'Length_Miles', 'ConnectorNetwork', 'CartoCode', 'AADT', 'AADT_YR',
       'BIKE_L', 'BIKE_R', 'VERT_LEVEL', 'PLANNED', 'SHAPE']

In [43]:
pd.concat([base_bb, bb_new_routes]).spatial.to_featureclass(location=os.path.join(gdb, 'mmn_bb'),sanitize_columns=False) 
pd.concat([planned_bb, bb_new_routes]).spatial.to_featureclass(location=os.path.join(gdb, 'mmn_planned_bb'),sanitize_columns=False) 

'e:\\Tasks\\Bike_Modeling_Data_Update\\Processing\\Outputs\\bb_modeling_20240911.gdb\\mmn_planned_bb'